# Ciência de Dados na Nuvem: O caminho do "Azure ML SDK"

## Introdução

Neste notebook, vamos aprender a usar o Azure ML SDK para treinar, implementar e consumir um modelo através do Azure ML.

Pré-requisitos:  
1. Criou um workspace do Azure ML.  
2. Carregou o [conjunto de dados de Insuficiência Cardíaca](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) no Azure ML.  
3. Carregou este notebook no Azure ML Studio.  

Os próximos passos são:

1. Criar uma Experiência num Workspace existente.  
2. Criar um cluster de Computação.  
3. Carregar o conjunto de dados.  
4. Configurar o AutoML usando AutoMLConfig.  
5. Executar a experiência AutoML.  
6. Explorar os resultados e obter o melhor modelo.  
7. Registar o melhor modelo.  
8. Implementar o melhor modelo.  
9. Consumir o endpoint.  

## Importações específicas do Azure Machine Learning SDK  


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Inicializar Espaço de Trabalho
Inicialize um objeto de espaço de trabalho a partir de uma configuração persistente. Certifique-se de que o ficheiro de configuração está presente em .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Criar uma experiência no Azure ML

Vamos criar uma experiência chamada 'aml-experiment' no espaço de trabalho que acabámos de inicializar.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Criar um Cluster de Computação
Será necessário criar um [alvo de computação](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) para a sua execução do AutoML.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Dados
Certifique-se de que carregou o conjunto de dados para o Azure ML e que a chave tem o mesmo nome que o conjunto de dados.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Configuração AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Execução AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Implementar o Melhor Modelo

Execute o seguinte código para implementar o melhor modelo. Pode verificar o estado da implementação no portal do Azure ML. Este passo pode demorar alguns minutos.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Consumir o Endpoint
Pode adicionar entradas ao exemplo de entrada abaixo.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução automática [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas resultantes do uso desta tradução.
